<a href="https://colab.research.google.com/github/Dimasputraangrh/bunga/blob/main/DESICION%20TREE%20FIX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix

TAMPILKAN DATASET

In [2]:
# Load dataset
data = pd.read_csv("/content/youtube-top-100-songs-2025.csv")

# Lihat 20 data pertama
display(data.head(20))

# Cek struktur data
data.info()

,title,fulltitle,description,view_count,categories,tags,duration,duration_string,live_status,thumbnail,channel,channel_url,channel_follower_count
0,ROSÉ & Bruno Mars - APT. (Official Music Video),ROSÉ & Bruno Mars - APT. (Official Music Video),ROSÉ & Bruno Mars - APT.\nDownload/stream: ht...,2009014557,Music,YG Entertainment;YG;와이지;K-pop;BLACKPINK;블랙핑크;블...,173,2:53,False,https://i.ytimg.com/vi_webp/ekr2nIex040/maxres...,ROSÉ,https://www.youtube.com/channel/UCBo1hnzxV9rz3...,19200000
1,"Lady Gaga, Bruno Mars - Die With A Smile (Offi...","Lady Gaga, Bruno Mars - Die With A Smile (Offi...",MAYHEM OUT NOW\nhttp://ladygaga.com \n \nListe...,1324833300,Music,Lady Gaga;Bruno Mars;Interscope;Pop,252,4:12,False,https://i.ytimg.com/vi/kPa7bsKwL-c/maxresdefau...,Lady Gaga,https://www.youtube.com/channel/UC07Kxew-cMIay...,29600000
2,Reneé Rapp - Leave Me Alone (Official Music Vi...,Reneé Rapp - Leave Me Alone (Official Music Vi...,"Listen to “BITE ME”, the new album from Reneé ...",2536628,Music,Reneé Rapp;Interscope Records;Pop,160,2:40,False,https://i.ytimg.com/vi/tiPWzFLiz4A/maxresdefau...,Reneé Rapp,https://www.youtube.com/channel/UCZy4ki_L4bzw9...,408000
3,Billie Eilish - BIRDS OF A FEATHER (Official M...,Billie Eilish - BIRDS OF A FEATHER (Official M...,Listen to HIT ME HARD AND SOFT: https://billie...,558329099,Music,Billie Eilish;Darkroom/Interscope Records;Alte...,231,3:51,False,https://i.ytimg.com/vi/V9PVRfjEBTI/maxresdefau...,Billie Eilish,https://www.youtube.com/channel/UCDGmojLIoWpXo...,56800000
4,Reneé Rapp - Mad (Official Music Video),Reneé Rapp - Mad (Official Music Video),"Listen to “BITE ME”, the new album from Reneé ...",2113548,Music,Reneé Rapp;Interscope Records;Pop,180,3:00,False,https://i.ytimg.com/vi/xkWQM3flsiY/maxresdefau...,Reneé Rapp,https://www.youtube.com/channel/UCZy4ki_L4bzw9...,408000
5,Sabrina Carpenter - Espresso,Sabrina Carpenter - Espresso,Music video by Sabrina Carpenter performing Es...,472570966,Music,Sabrina Carpenter;Island Records;Pop,201,3:21,False,https://i.ytimg.com/vi/eVli-tstM5E/maxresdefau...,Sabrina Carpenter,https://www.youtube.com/channel/UC4mYKePIas5Yg...,12300000
6,Lady Gaga - Abracadabra (Official Music Video),Lady Gaga - Abracadabra (Official Music Video),MAYHEM OUT NOW\nhttp://ladygaga.com\n \nDirect...,191073418,Music,Lady Gaga;Interscope Records;Pop;lady gaga abr...,269,4:29,False,https://i.ytimg.com/vi/vBynw9Isr28/maxresdefau...,Lady Gaga,https://www.youtube.com/channel/UC07Kxew-cMIay...,29600000
7,Ed Sheeran - Sapphire (Official Music Video),Ed Sheeran - Sapphire (Official Music Video),Listen to 'Sapphire' now: https://es.lnk.to/sa...,184696317,Music,edsheeran;ed sheeran;acoustic;live;cover;offic...,183,3:03,False,https://i.ytimg.com/vi_webp/JgDNFQ2RaLQ/maxres...,Ed Sheeran,https://www.youtube.com/channel/UC0C-w0YjGpqDX...,58500000
8,yung kai - blue (official music video),yung kai - blue (official music video),Stream my track and follow me on socials: http...,187281056,People & Blogs,yung kai;blue;tiktok viral;tik tok;wave to ear...,221,3:41,False,https://i.ytimg.com/vi_webp/IpFX2vq8HKw/maxres...,yung kai,https://www.youtube.com/channel/UCy1FLad-XXw7Q...,1220000
9,Billie Eilish - WILDFLOWER (BILLIE BY FINNEAS),Billie Eilish - WILDFLOWER (BILLIE BY FINNEAS),HIT ME HARD AND SOFT (WITH ALL ISOLATED VOCALS...,40408980,Music,Billie Eilish;Darkroom/Interscope Records;Alte...,261,4:21,False,https://i.ytimg.com/vi/IWKyykQBRKo/maxresdefau...,Billie Eilish,https://www.youtube.com/channel/UCDGmojLIoWpXo...,56800000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   title                   100 non-null    object
 1   fulltitle               100 non-null    object
 2   description             100 non-null    object
 3   view_count              100 non-null    int64 
 4   categories              100 non-null    object
 5   tags                    85 non-null     object
 6   duration                100 non-null    int64 
 7   duration_string         100 non-null    object
 8   live_status             100 non-null    bool  
 9   thumbnail               100 non-null    object
 10  channel                 100 non-null    object
 11  channel_url             100 non-null    object
 12  channel_follower_count  100 non-null    int64 
dtypes: bool(1), int64(3), object(9)
memory usage: 9.6+ KB


KATEGORIKAN

In [3]:
df = pd.read_csv("/content/youtube-top-100-songs-2025.csv")

# Kategori durasi
def kategori_durasi(d):
    if d < 180:
        return "Pendek"
    elif d <= 300:
        return "Sedang"
    else:
        return "Panjang"

df['durasi_kategori'] = df['duration'].apply(kategori_durasi)

# Kelas view
conditions = [
    df['view_count'] < 100_000_000,
    (df['view_count'] >= 100_000_000) & (df['view_count'] < 1_000_000_000),
    df['view_count'] >= 1_000_000_000
]
choices = ['Rendah', 'Sedang', 'Tinggi']
df['kelas_view'] = np.select(conditions, choices, default='')

ENCODING

In [4]:
X = df[['categories', 'durasi_kategori', 'channel', 'channel_follower_count']]
y = df['kelas_view']

# Encode output
label_y = LabelEncoder()
y_enc = label_y.fit_transform(y)

# One-Hot Encoding input
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

encoder = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'),
         ['categories', 'durasi_kategori', 'channel']),
        ('num', 'passthrough', ['channel_follower_count'])
    ]
)

X_encoded = encoder.fit_transform(X)
# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y_enc, test_size=0.2, random_state=42
)

TRAIN DECISION TREE

In [7]:
model = DecisionTreeClassifier(
    criterion='entropy',
    max_depth=10,
    random_state=42
)

model.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=10, random_state=42)

EVALUASI MODEL

In [8]:
y_pred = model.predict(X_test)

print(classification_report(
    y_test, y_pred,
    target_names=label_y.classes_
))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

      Rendah       0.81      0.87      0.84        15
      Sedang       0.25      0.25      0.25         4
      Tinggi       0.00      0.00      0.00         1

    accuracy                           0.70        20
   macro avg       0.35      0.37      0.36        20
weighted avg       0.66      0.70      0.68        20

Confusion Matrix:
[[13  2  0]
 [ 3  1  0]
 [ 0  1  0]]


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


PREDIKSI MANUAL

In [10]:
print("=== INPUT MANUAL DECISION TREE ===")

kategori = input("Masukkan kategori musik: ").title()
durasi = input("Masukkan durasi (Pendek/Sedang/Panjang): ").title()
penyanyi = input("Masukkan nama penyanyi (1 nama): ").title()
subscriber = int(input("Masukkan jumlah follower channel (angka): ")) # Added line to get subscriber input

input_df = pd.DataFrame({
    'categories': [kategori],
    'durasi_kategori': [durasi],
    'channel': [penyanyi],
    'channel_follower_count': [subscriber]
})

input_enc = encoder.transform(input_df)

pred = model.predict(input_enc)
prob = model.predict_proba(input_enc)

kelas = label_y.inverse_transform(pred)

print("\nHasil Prediksi View:", kelas[0])
print("\nProbabilitas:")
for i, k in enumerate(label_y.classes_):
    print(f"{k}: {prob[0][i]*100:.2f}%")

=== INPUT MANUAL DECISION TREE ===
Masukkan kategori musik: musik
Masukkan durasi (Pendek/Sedang/Panjang): sedang
Masukkan nama penyanyi (1 nama): Teddy Swims - Bad Dreams
Masukkan jumlah follower channel (angka): 6070000

Hasil Prediksi View: Rendah

Probabilitas:
Rendah: 81.82%
Sedang: 18.18%
Tinggi: 0.00%
